<a href="https://colab.research.google.com/github/apotheosis-ai/MusicLSTM/blob/main/MusicLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Importing Required Packages**


In [4]:
!pip install mido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
import mido
import tensorflow as tf
from keras import backend as k
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
# from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np

##**Load MID file/files**

In [6]:
!unzip /content/chillhopdata.zip -d chillhop

Archive:  /content/chillhopdata.zip
  inflating: chillhop/1.mid          
  inflating: chillhop/10.mid         
  inflating: chillhop/11.mid         
  inflating: chillhop/12.mid         
  inflating: chillhop/13.mid         
  inflating: chillhop/14.mid         
  inflating: chillhop/15.mid         
  inflating: chillhop/16.mid         
  inflating: chillhop/17.mid         
  inflating: chillhop/18.mid         
  inflating: chillhop/19.mid         
  inflating: chillhop/2.mid          
  inflating: chillhop/20.mid         
  inflating: chillhop/3.mid          
  inflating: chillhop/4.mid          
  inflating: chillhop/5.mid          
  inflating: chillhop/6.mid          
  inflating: chillhop/7.mid          
  inflating: chillhop/8.mid          
  inflating: chillhop/9.mid          
  inflating: chillhop/Cymatics - Eternity MIDI 1 - C Maj.mid  
  inflating: chillhop/Cymatics - Eternity MIDI 10 - F Min.mid  
  inflating: chillhop/Cymatics - Eternity MIDI 11 - A Maj.mid  
  inflating: 

In [24]:
import os

notes = []
for song in os.listdir("/content/chillhop"):
  mid = MidiFile('/content/chillhop/' + song)
  print(mid)

  for msg in mid:
    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
      data = msg.bytes()
      notes.append(data[1])

Streaming output truncated to the last 5000 lines.
    Message('note_off', channel=0, note=76, velocity=49, time=163),
    Message('note_on', channel=0, note=81, velocity=84, time=26),
    Message('note_on', channel=0, note=83, velocity=82, time=45),
    Message('note_off', channel=0, note=81, velocity=90, time=1),
    Message('note_off', channel=0, note=69, velocity=96, time=48),
    Message('note_off', channel=0, note=83, velocity=127, time=2),
    Message('note_on', channel=0, note=84, velocity=93, time=0),
    Message('note_off', channel=0, note=67, velocity=37, time=2),
    Message('note_off', channel=0, note=64, velocity=71, time=1),
    Message('note_off', channel=0, note=53, velocity=64, time=3),
    Message('note_off', channel=0, note=60, velocity=66, time=3),
    Message('note_off', channel=0, note=84, velocity=92, time=5),
    Message('note_on', channel=0, note=81, velocity=81, time=29),
    Message('note_on', channel=0, note=62, velocity=56, time=5),
    Message('note_on', 

##**Scale Data**

In [25]:
scaler = MinMaxScaler()
notes = list(scaler.fit_transform(np.array(notes).reshape(-1,1)))

In [26]:
notes

[array([0.35483871]),
 array([0.48387097]),
 array([0.38709677]),
 array([0.15053763]),
 array([0.31182796]),
 array([0.27956989]),
 array([0.51612903]),
 array([0.53763441]),
 array([0.55913978]),
 array([0.48387097]),
 array([0.51612903]),
 array([0.53763441]),
 array([0.55913978]),
 array([0.48387097]),
 array([0.51612903]),
 array([0.53763441]),
 array([0.55913978]),
 array([0.59139785]),
 array([0.55913978]),
 array([0.53763441]),
 array([0.51612903]),
 array([0.55913978]),
 array([0.30107527]),
 array([0.09677419]),
 array([0.22580645]),
 array([0.33333333]),
 array([0.25806452]),
 array([0.15053763]),
 array([0.22580645]),
 array([0.25806452]),
 array([0.31182796]),
 array([0.33333333]),
 array([0.61290323]),
 array([0.38709677]),
 array([0.48387097]),
 array([0.35483871]),
 array([0.51612903]),
 array([0.69892473]),
 array([0.68817204]),
 array([0.56989247]),
 array([0.59139785]),
 array([0.07526882]),
 array([0.15053763]),
 array([0.1827957]),
 array([0.23655914]),
 array([0.2

##**Create Train Data**

In [27]:
# Predict next nte for first 30 notes

notes = [list(note) for note in notes]

X = []
y = []

n_prev=30

for i in range(len(notes) - n_prev):
  X.append(notes[i:i+n_prev])
  y.append(notes[i+n_prev])

X_test = X[-300:]
X = X[:-300]
y = y[:-300]

##**Build LSTM**

In [32]:
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.6))
model.add(Dense(1))
model.add(Activation('linear'))
model.summary()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mse', optimizer=optimizer)


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 30, 256)           264192    
                                                                 
 dropout_12 (Dropout)        (None, 30, 256)           0         
                                                                 
 lstm_13 (LSTM)              (None, 30, 128)           197120    
                                                                 
 dropout_13 (Dropout)        (None, 30, 128)           0         
                                                                 
 lstm_14 (LSTM)              (None, 64)                49408     
                                                                 
 dropout_14 (Dropout)        (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                

##**Training**

In [35]:
model.fit(np.array(X), np.array(y), batch_size=16, epochs=10, verbose=1)

Epoch 1/10
195/195 [==============================] - 2s 9ms/step - loss: 0.0151
Epoch 2/10
195/195 [==============================] - 2s 9ms/step - loss: 0.0151
Epoch 3/10
195/195 [==============================] - 2s 9ms/step - loss: 0.0151
Epoch 4/10
195/195 [==============================] - 2s 9ms/step - loss: 0.0149
Epoch 5/10
195/195 [==============================] - 2s 9ms/step - loss: 0.0148
Epoch 6/10
195/195 [==============================] - 2s 9ms/step - loss: 0.0146
Epoch 7/10
195/195 [==============================] - 2s 9ms/step - loss: 0.0150
Epoch 8/10
195/195 [==============================] - 2s 9ms/step - loss: 0.0149
Epoch 9/10
195/195 [==============================] - 2s 10ms/step - loss: 0.0148
Epoch 10/10
195/195 [==============================] - 2s 9ms/step - loss: 0.0147


##**Generating & Saving LSTM Music**

In [36]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]

mid = MidiFile()
track = MidiTrack()
t = 0
for note in prediction:
    # 147 means note_on
    # 67 is velosity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('LSTM_music.mid')

10/10 [==============================] - 0s 5ms/step
